In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3

In [2]:


hard = pd.read_csv("hard23.csv")
clay = pd.read_csv("clay23.csv")
grass = pd.read_csv("grass23.csv")
overall23 = pd.read_csv("2023overall.csv")

#Renaming the columns
# hard = [df.rename(columns={"value": "Hard_Win%"}) for df in hard]
# clay = [df.rename(columns={"value": "Clay_Win%"}) for df in clay]
# grass = [df.rename(columns={"value": "Grass_Win%"}) for df in grass]

hard = hard.rename(columns = {"value": "hard_Win%"})
clay = clay.rename(columns = {"value": "clay_Win%"})
grass = grass.rename(columns = {"value": "grass_Win%"})

overall23 = overall23.rename(columns = {"value": "Overall_Win%"})


# get tennis abstract dataset and see what the intersecting dataframes look like for hard, clay, and grass
clayserve = pd.read_csv("clayserve.csv")
hardserve = pd.read_csv("hardserve.csv")
grassserve = pd.read_csv("grassserve.csv")
clayreturn = pd.read_csv("clayreturn.csv")
hardreturn = pd.read_csv("hardreturn.csv")
grassreturn = pd.read_csv("grassreturn.csv")
rally = pd.read_csv("rally.csv", encoding='windows-1252')

#edit player names to get rid of the country at end 
hardserve['Player'] = hardserve['Player'].str.slice(0,-6)
clayserve['Player'] = clayserve['Player'].str.slice(0,-6)
grassserve['Player'] = grassserve['Player'].str.slice(0,-6)
hardreturn['Player'] = clayreturn['Player'].str.slice(0,-6)
clayreturn['Player'] = clayreturn['Player'].str.slice(0,-6)
grassreturn['Player'] = clayreturn['Player'].str.slice(0,-6)
rally['Player'] = rally['Player'].str.replace('\xa0',' ', regex = False)

#return

# hard = [hard21, hard22, hard23]
# clay = [clay21, clay22, clay23]
# grass = [grass21, grass22, grass23]

In [3]:
#rename fritz alcaraz rune 
def rename(df):
    df['name'] = df['name'].replace('Carlos Alcaraz Garfia', 'Carlos Alcaraz')
    df['name'] = df['name'].replace('Taylor Harry Fritz', 'Taylor Fritz')
    df['name'] = df['name'].replace('Holger Vitus Nodskov Rune', 'Holger Rune')
rename(hard)
rename(clay)
rename(grass)

In [4]:
#Now, let's merge all the 21 dfs together?
conn = sqlite3.connect("tennis.db")
hard.to_sql("hard", conn, if_exists = 'append', index = False)
clay.to_sql("clay", conn, if_exists = 'append', index = False)
grass.to_sql("grass", conn, if_exists = 'append', index = False)
hardserve.to_sql("hardserve", conn, if_exists = 'append', index = False)
clayserve.to_sql("clayserve", conn, if_exists = 'append', index = False)
grassserve.to_sql("grassserve", conn, if_exists = 'append', index = False)
hardreturn.to_sql("hardreturn", conn, if_exists = 'append', index = False)
clayreturn.to_sql("clayreturn", conn, if_exists = 'append', index = False)
grassreturn.to_sql("grassreturn", conn, if_exists = 'append', index = False)
rally.to_sql("rally", conn, if_exists = 'append', index = False)
conn.close()


In [5]:
# conn = sqlite3.connect("tennis.db")
# cursor = conn.cursor()

# #sql query
# cmd = \
# """ 
# SELECT H.name, H.country_name, H.'Hard_Win%', C.'Clay_Win%', G.'Grass_Win%'
# FROM hard H
# INNER JOIN clay C ON H.name = C.name
# INNER JOIN grass G on H.name = G.name;
# """
# tentative = pd.read_sql_query(cmd, conn) #read the query to a dataframe
# conn.close()
# tentative

# Functions for serve, return, and rally

In [8]:
def serve(surface):
    '''
    sql query that returns serve dataframe for specified surface
    '''
    conn = sqlite3.connect("tennis.db")

    #sql query
    cmd = \
    f""" 
    SELECT H.name, H.country_name, H.'{surface}_Win%', HS.'Ace%', HS.SPW , HS.'DF%', HS.'1st%', HS.'2nd%', HS.'Hld%', HS.'Pts/SG' 
    FROM {surface} H
    INNER JOIN {surface}serve HS ON H.name = HS.player
    """
    serve = pd.read_sql_query(cmd, conn)
    conn.close()
    return serve

In [11]:
def returns(surface):
    '''sql query that returns return dataframe for specified surface'''
    conn = sqlite3.connect("tennis.db")

    #sql query
    cmd = \
    f""" 
    SELECT H.name, H.country_name, H.'{surface}_Win%', HS.RPW, HS.'vAce%' , HS.'v1st%', HS.'v2nd%', HS.'Brk%', HS.'PtsW/RG'
    FROM {surface} H
    INNER JOIN {surface}return HS ON H.name = HS.player
    """
    returns = pd.read_sql_query(cmd, conn)
    conn.close()
    return returns

In [16]:
def rally(surface):
    conn = sqlite3.connect("tennis.db")
    #sql query
    cmd = \
    f""" 
    SELECT H.name, H.country_name, H.'{surface}_Win%', HS.RallyLen, HS.'RLen-Serve' , HS.'RLen-Return',
    HS.'1-3\xa0W%', HS.'4-6\xa0W%', HS.'7-9\xa0W%', HS.'10+\xa0W%', HS.'FH/GS', HS.'BH\xa0Slice%', HS.'FHP/Match', HS.'BHP/Match'
    FROM {surface} H
    INNER JOIN rally HS ON H.name = HS.Player
    """
    rally = pd.read_sql_query(cmd, conn)
    conn.close()
    return rally

In [17]:
hardserve = serve("hard")
clayserve = serve("clay")
grassserve = serve("grass")

In [18]:
hardreturns = returns("hard")
clayreturns = returns("clay")
grassreturns = returns("grass")

In [19]:
hardrally = rally("hard")
clayrally = rally("clay")
grassrally = rally("grass")

In [20]:
hardrally

,name,country_name,hard_Win%,RallyLen,RLen-Serve,RLen-Return,1-3 W%,4-6 W%,7-9 W%,10+ W%,FH/GS,BH Slice%,FHP/Match,BHP/Match
0,Novak Djokovic,Serbia,96.43%,4.6,4.5,4.7,54.00%,56.90%,53.90%,54.60%,50.20%,16.80%,14.5,5.1
1,Daniil Medvedev,Russian Federation,83.93%,5.0,4.5,5.5,53.30%,53.70%,51.60%,53.70%,45.30%,8.40%,9.0,6.0
2,Jannik Sinner,Italy,83.33%,4.3,4.4,4.2,54.10%,53.70%,51.80%,53.40%,51.20%,6.30%,7.3,1.2
3,Carlos Alcaraz,Spain,81.25%,3.9,3.9,3.9,51.40%,56.30%,54.90%,56.20%,51.90%,18.20%,10.1,2.2
4,Taylor Fritz,United States,74.51%,4.3,4.3,4.2,50.70%,54.10%,49.30%,46.00%,51.30%,5.60%,9.7,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Marc Andrea Huesler,Switzerland,37.50%,3.6,3.5,3.8,48.00%,45.80%,34.30%,41.00%,55.60%,18.00%,2.4,-4.6
61,Jan Lennard Struff,Germany,36.36%,3.0,3.0,2.9,48.60%,51.90%,52.30%,53.30%,44.90%,10.60%,6.1,-0.5
62,Christopher Oconnell,Australia,35.29%,3.8,3.8,3.8,51.40%,48.90%,50.70%,40.80%,52.40%,54.90%,3.8,0.1
63,Roberto Carballes Baena,Spain,30.00%,5.2,5.6,4.8,49.10%,50.30%,46.30%,47.90%,54.80%,11.30%,11.9,1.0
